# Gensim Tutorial 
## Confirmation of LSA algorithm
* Gensim official tutorial
    * http://radimrehurek.com/gensim/tutorial.html]
* Gensim tutorial (Japanese)
    * Corupus and vector space http://hivecolor.com/id/58
    * Topics and transformation http://hivecolor.com/id/59
    * Similarity queries http://hivecolor.com/id/60
---

In [32]:
#import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

from gensim import corpora, models, similarities

documents = ["Human machine interface for lab abc computer applications",
             "A survey of user opinion of computer system response time",
             "The EPS user interface management system",
             "System and human system engineering testing of EPS",
             "Relation of user perceived response time to error measurement",
             "The generation of random binary unordered trees",
             "The intersection graph of paths in trees",
             "Graph minors IV Widths of trees and well quasi ordering",
             "Graph minors A survey"]
# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
          for document in documents]
#pprint(texts)
#pprint("-------------")

# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text: 
         frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1]
          for text in texts]

from pprint import pprint  # pretty-printer
pprint(texts)

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


In [33]:
dictionary = corpora.Dictionary(texts)
#dictionary.save('/deerwester.dict')  # store the dictionary, for future reference
# バイナリではなくテキストとして保存する場合
dictionary.save_as_text('.\deerwester_text.dict')
print(dictionary)

#各トークンのIDを表示
print (dictionary.token2id)

2017-01-08 00:58:53,454 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2017-01-08 00:58:53,470 : INFO : built Dictionary(12 unique tokens: ['survey', 'interface', 'computer', 'graph', 'user']...) from 9 documents (total 29 corpus positions)
2017-01-08 00:58:53,470 : INFO : saving dictionary mapping to .\deerwester_text.dict


Dictionary(12 unique tokens: ['survey', 'interface', 'computer', 'graph', 'user']...)
{'survey': 3, 'interface': 0, 'computer': 1, 'graph': 10, 'user': 7, 'time': 5, 'response': 6, 'system': 4, 'minors': 11, 'human': 2, 'trees': 9, 'eps': 8}


In [34]:
# doc2bow()はdistinctした結果から単語の出現回数を数えます。
#[(0, 1), (1, 1)]という出力結果から、computerの辞書内のIDは0で今回の出現回数1回、
#humanの辞書内のIDは1で今回の出現回数は1回、interctionは辞書に入っていない(なので無視された)、ということが分かります。
new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print (new_vec) # the word "interaction" does not appear in the dictionary and is ignored

corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('.\deerwester.mm', corpus) # store to disk, for later use
corpus = corpora.MmCorpus('.\deerwester.mm')

# print list(corpus)
print (corpus)
print ("-------------------tutorial2")
tfidf = models.TfidfModel(corpus) # step 1 -- initialize a model
print (new_vec)
print (tfidf[new_vec]) # step 2 -- use the model to transform vectors
print ("---tfidf---------------")
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
    print (doc)
  

2017-01-08 00:58:53,548 : INFO : storing corpus in Matrix Market format to .\deerwester.mm
2017-01-08 00:58:53,564 : INFO : saving sparse matrix to .\deerwester.mm
2017-01-08 00:58:53,564 : INFO : PROGRESS: saving document #0
2017-01-08 00:58:53,564 : INFO : saved 9x12 matrix, density=25.926% (28/108)
2017-01-08 00:58:53,579 : INFO : saving MmCorpus index to .\deerwester.mm.index


[(1, 1), (2, 1)]


2017-01-08 00:58:53,595 : INFO : loaded corpus index from .\deerwester.mm.index
2017-01-08 00:58:53,595 : INFO : initializing corpus reader from .\deerwester.mm
2017-01-08 00:58:53,595 : INFO : accepted corpus with 9 documents, 12 features, 28 non-zero entries
2017-01-08 00:58:53,611 : INFO : collecting document frequencies
2017-01-08 00:58:53,611 : INFO : PROGRESS: processing document #0
2017-01-08 00:58:53,611 : INFO : calculating IDF weights for 9 documents and 11 features (28 matrix non-zeros)


MmCorpus(9 documents, 12 features, 28 non-zero entries)
-------------------tutorial2
[(1, 1), (2, 1)]
[(1, 0.7071067811865476), (2, 0.7071067811865476)]
---tfidf---------------
[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)]
[(1, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.3244870206138555), (5, 0.44424552527467476), (6, 0.44424552527467476), (7, 0.3244870206138555)]
[(0, 0.5710059809418182), (4, 0.4170757362022777), (7, 0.4170757362022777), (8, 0.5710059809418182)]
[(2, 0.49182558987264147), (4, 0.7184811607083769), (8, 0.49182558987264147)]
[(5, 0.6282580468670046), (6, 0.6282580468670046), (7, 0.45889394536615247)]
[(9, 1.0)]
[(9, 0.7071067811865475), (10, 0.7071067811865475)]
[(9, 0.5080429008916749), (10, 0.5080429008916749), (11, 0.695546419520037)]
[(3, 0.6282580468670046), (10, 0.45889394536615247), (11, 0.6282580468670046)]


In [35]:
  
print ("---lsi---------------")   
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2) # initialize an LSI transformation
corpus_lsi = lsi[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi
lsi.print_topics(2)

for doc in corpus_lsi: # both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
    print (doc)
    
lsi.save('.\model.lsi') # same for tfidf, lda, ...
lsi = models.LsiModel.load('.\model.lsi')

2017-01-08 00:58:53,673 : INFO : using serial LSI version on this node


---lsi---------------


2017-01-08 00:58:53,673 : INFO : updating model with new documents
2017-01-08 00:58:53,689 : INFO : preparing a new chunk of documents
2017-01-08 00:58:53,689 : INFO : using 100 extra samples and 2 power iterations
2017-01-08 00:58:53,689 : INFO : 1st phase: constructing (12, 102) action matrix
2017-01-08 00:58:53,689 : INFO : orthonormalizing (12, 102) action matrix
2017-01-08 00:58:53,704 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2017-01-08 00:58:53,704 : INFO : computing the final decomposition
2017-01-08 00:58:53,704 : INFO : keeping 2 factors (discarding 47.565% of energy spectrum)
2017-01-08 00:58:53,704 : INFO : processed documents up to #9
2017-01-08 00:58:53,720 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"response" + 0.060*"time" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2017-01-08 00:58:53,720 : INFO : topic #1(1.476): -0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"inte

[(0, 0.066007833960905024), (1, -0.5200703306361848)]
[(0, 0.19667592859142788), (1, -0.76095631677000386)]
[(0, 0.089926399724467129), (1, -0.72418606267525054)]
[(0, 0.075858476521784346), (1, -0.63205515860034267)]
[(0, 0.10150299184980362), (1, -0.57373084830029486)]
[(0, 0.70321089393783065), (1, 0.16115180214026062)]
[(0, 0.8774787673119826), (1, 0.16758906864659731)]
[(0, 0.90986246868185727), (1, 0.14086553628719348)]
[(0, 0.61658253505692828), (1, -0.053929075663891483)]


2017-01-08 00:58:53,767 : INFO : saved .\model.lsi.projection
2017-01-08 00:58:53,767 : INFO : saving LsiModel object under .\model.lsi, separately None
2017-01-08 00:58:53,767 : INFO : not storing attribute projection
2017-01-08 00:58:53,783 : INFO : not storing attribute dispatcher
2017-01-08 00:58:53,783 : INFO : saved .\model.lsi
2017-01-08 00:58:53,783 : INFO : loading LsiModel object from .\model.lsi
2017-01-08 00:58:53,798 : INFO : loading id2word recursively from .\model.lsi.id2word.* with mmap=None
2017-01-08 00:58:53,798 : INFO : setting ignored attribute projection to None
2017-01-08 00:58:53,798 : INFO : setting ignored attribute dispatcher to None
2017-01-08 00:58:53,798 : INFO : loaded .\model.lsi
2017-01-08 00:58:53,798 : INFO : loading LsiModel object from .\model.lsi.projection
2017-01-08 00:58:53,814 : INFO : loaded .\model.lsi.projection
